In [1]:
import os
import shutil

import re
import time
import pandas as pd
import numpy as np

import pprint
from ast import literal_eval


Corpus = pd.read_excel('CorpusBeta.xlsx', header=0)
Corpus['From-->Too'] = Corpus['From-->Too'].apply(literal_eval)
Corpus['Method Used'] = Corpus['Method Used'].apply(literal_eval)

print('la taille de la corpus : '+str(len(Corpus)))
Corpus.head()

la taille de la corpus : 2014


,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,"[persian, arabic, english]",[statistical],[]
1,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,"[arabic, english]",[statistical],[]
2,An automatic and a machine-assisted method to ...,Two different methods of corpus cleaning are p...,Bilingual corpus cleaning; Statistical machine...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3342351,"Srivastava J., Sanyal S., Srivastava A.K.",Article,Scopus,"[hindi, english]",[statistical],[]
3,Combining MEDLINE and publisher data to create...,Background: Most of the institutional and rese...,Biomedical domain; Multilingual corpus generat...,https://www.scopus.com/inward/record.uri?eid=2...,2013,14.0,NaN,NaN,1,10.1186/1471-2105-14-146,"Jimeno Yepes A., Prieur-Gaston É., Névéol A.",Article,Scopus,"[spanish, french, english]",[statistical],[]
4,Syntax-based Chinese-Vietnamese tree-to-tree s...,"Because of the scarcity of bilingual corpora, ...",Chinese-Vietnamese; Linguistic features; Pruni...,https://www.scopus.com/inward/record.uri?eid=2...,2019,1.0,NaN,NaN,1,10.1145/3314938,"Gao S., Huang J., Xue M., Yu Z., Wang Z., Zhan...",Article,Scopus,"[vietnamese, chinese]",[statistical],[]


### Languages Separation 

In [2]:
LongCorpusLang = Corpus.explode('From-->Too').reset_index(drop=True)
print('the lenght of the corpus after separating the languages are : '+str(len(LongCorpusLang)))
LongCorpusLang.head()

the lenght of the corpus after separating the languages are : 3434


,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,persian,[statistical],[]
1,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,arabic,[statistical],[]
2,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,english,[statistical],[]
3,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,arabic,[statistical],[]
4,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,english,[statistical],[]


In [3]:
LongCorpusLang.drop(LongCorpusLang.index[(LongCorpusLang['Year'] == 2021.0) | (LongCorpusLang['Year'] == 2022.0)], inplace = True)
LongCorpusLang['From-->Too'].value_counts()[:10]

english     876
chinese     326
german      121
japanese    115
arabic      105
hindi        99
french       96
spanish      89
russian      44
korean       41
Name: From-->Too, dtype: int64

In [4]:
LongCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'english') | (LongCorpusLang.astype(str)['From-->Too'] == 'arabic') | (LongCorpusLang.astype(str)['From-->Too'] == 'chinese') | (LongCorpusLang.astype(str)['From-->Too'] == 'german') | (LongCorpusLang.astype(str)['From-->Too'] == 'japanese') | (LongCorpusLang.astype(str)['From-->Too'] == 'hindi') | (LongCorpusLang.astype(str)['From-->Too'] == 'french') | (LongCorpusLang.astype(str)['From-->Too'] == 'spanish') | (LongCorpusLang.astype(str)['From-->Too'] == 'russian') | (LongCorpusLang.astype(str)['From-->Too'] == 'korean')]

LongCorpusLang.reset_index(drop=True, inplace=True)
LongCorpusLang
#LongCorpusLang.to_excel('LongCorpusLang.xlsx', index=False)

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,arabic,[statistical],[]
1,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,english,[statistical],[]
2,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,arabic,[statistical],[]
3,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,english,[statistical],[]
4,An automatic and a machine-assisted method to ...,Two different methods of corpus cleaning are p...,Bilingual corpus cleaning; Statistical machine...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3342351,"Srivastava J., Sanyal S., Srivastava A.K.",Article,Scopus,hindi,[statistical],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,Human evaluation of online machine translation...,This paper presents results of human evaluatio...,Adequacy; Croatian; English; Error analysis; F...,https://www.scopus.com/inward/record.uri?eid=2...,2015,5.0,1089,1098,10,10.1007/978-3-319-16486-1_108,"Seljan S., Tucakovi? M., Dun?er I.",Conference Paper,Scopus,english,[],[]
1908,Machine translation architecture based on micr...,This paper surveys current approaches of multi...,Machine translation; Micro-engine pipeline; Mu...,https://www.scopus.com/inward/record.uri?eid=2...,2004,NaN,651,658,8,NaN,Liu Q.,Article,Scopus,chinese,[],[]
1909,Machine translation architecture based on micr...,This paper surveys current approaches of multi...,Machine translation; Micro-engine pipeline; Mu...,https://www.scopus.com/inward/record.uri?eid=2...,2004,NaN,651,658,8,NaN,Liu Q.,Article,Scopus,english,[],[]
1910,Uyghur-Chinese translation disambiguation meth...,This thesis studies the disambiguation method ...,Knowledge automatic-acquisition; Uyghur-chines...,https://www.scopus.com/inward/record.uri?eid=2...,2015,NaN,NaN,NaN,6,NaN,"Ge R., Yong Y., Chun X.",Article,Scopus,chinese,[],[]


In [5]:
EnglishCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'english')]
EnglishCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1024,The Multistore Parser for Hierarchical Syntact...,A syntactic parser is described for hierarchic...,automatic abstracting; computational linguisti...,https://www.scopus.com/inward/record.uri?eid=2...,1970,17.0,74,82,9,10.1145/362007.362026,"von Glasersfeld E., Pisani P.P.",Article,Scopus,english,[],[]
1086,An Intelligent Analyzer and Understander of En...,The paper describes a working analysis and gen...,artificial intelligence; computational linguis...,https://www.scopus.com/inward/record.uri?eid=2...,1975,73.0,264,274,11,10.1145/360762.360770,Wilks Y.,Article,Scopus,english,[],[]
1169,Steps Toward Knowledge-Based Machine Translation,This paper considers the possibilities for kno...,Artificial intelligence; computational linguis...,https://www.scopus.com/inward/record.uri?eid=2...,1981,17.0,376,392,17,10.1109/TPAMI.1981.4767124,"Carbonell J.G., Cullingford R.E., Gershman A.V.",Article,Scopus,english,[],[]
1040,The Automatic Inversion of Attribute Grammars,Over the last decade there has developed an ac...,Automatic software generation; bidirectional t...,https://www.scopus.com/inward/record.uri?eid=2...,1986,4.0,590,599,10,10.1109/TSE.1986.6312955,"Yellin D.M., Mueckstein E.-M.M.",Article,Scopus,english,[],[]
1212,The treatment of complex English nominalizatio...,"This paper concerns the treatment, in the cont...",Japanese; Machine translation; nominalization;...,https://www.scopus.com/inward/record.uri?eid=2...,1988,1.0,3,21,19,10.1007/BF02057966,"Somers H., Hirakawa H., Miike S., Amano S.",Article,Scopus,english,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,Neural Machine Translation Based on Improved A...,Reinforcement learning based neural machine tr...,Actor-Critic Method; Non-autoregressive neural...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,346,357,12,10.1007/978-3-030-61616-8_28,"Guo Z., Hou H., Wu N., Sun S.",Conference Paper,Scopus,english,[neural],[]
715,Quality Estimation for Machine Translation wit...,Quality estimation (QE) for machine translatio...,Multi-granularity; Neural machine translation;...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,55,65,11,10.1007/978-981-33-6162-1_5,"Tian K., Zhang J.",Conference Paper,Scopus,english,[neural],[]
1622,Dynamic Context-guided Capsule Network for Mul...,"Multimodal machine translation (MMT), which ma...",capsule network; multimodal machine translatio...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,1320,1329,10,10.1145/3394171.3413715,"Lin H., Meng F., Su J., Yin Y., Yang Z., Ge Y....",Conference Paper,Scopus,english,[],[]
815,Linguistically informed Hindi-english neural m...,Hindi-English Machine Translation is a challen...,Linguistic Features; Neural Machine Translatio...,https://www.scopus.com/inward/record.uri?eid=2...,2020,1.0,3698,3703,6,NaN,"Goyal V., Mishra P., Sharma D.M.",Conference Paper,Scopus,english,[neural],[]


In [6]:
ChineseCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'chinese')]
ChineseCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1213,"Semantic analysis in XTRA, an English - Chines...",This paper concerns the resolution of lexical ...,Chinese; integration of syntax and semantics; ...,https://www.scopus.com/inward/record.uri?eid=2...,1988,4.0,101,120,20,10.1007/BF02055234,Huang X.,Article,Scopus,chinese,[],[]
185,Some key issues in designing MT systems,Development of a machine translation system (M...,Chinese; control strategy; English; generative...,https://www.scopus.com/inward/record.uri?eid=2...,1990,9.0,265,300,36,10.1007/BF00376644,"Su K.-Y., Chang J.-S.",Article,Scopus,chinese,[statistical],[]
1266,Resolution of ambiguity in Chinese and its app...,"In this paper, we present a typology of ambigu...",ambiguity typology; Chinese; disambiguation; p...,https://www.scopus.com/inward/record.uri?eid=2...,1991,NaN,279,315,37,10.1007/BF00417653,"Chang C.-H., Krulee G.K.",Article,Scopus,chinese,[],[]
183,Large-scale automatic extraction of an English...,We report experimental results on automatic ex...,English-Chinese machine translation; lexical a...,https://www.scopus.com/inward/record.uri?eid=2...,1994,22.0,285,313,29,10.1007/BF00980581,"Wu D., Xia X.",Article,Scopus,chinese,[statistical],[]
1304,An automatic sign recognition and translation ...,A sign is something that suggests the presence...,Perceptive user interface; Sign detection; Sig...,https://www.scopus.com/inward/record.uri?eid=2...,2001,2.0,9,NaN,-8,10.1145/971478.971490,"Yang J., Gao J., Zhang Y., Chen X., Waibel A.",Conference Paper,Scopus,chinese,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1740,The Annotation Scheme of English-Chinese Claus...,"A clause complex consists of clauses, which ar...",Clause-complex; Corpus annotation; Machine tra...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,287,299,13,10.1007/978-3-030-63031-7_21,"Ge S., Lin X., Song R.",Conference Paper,Scopus,chinese,[],[]
1746,Ensemble Distilling Pretrained Language Models...,Machine translation quality estimation (Qualit...,Knowledge distillation; Machine translation; P...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,231,243,13,10.1007/978-3-030-60457-8_19,"Huang H., Di H., Xu J., Ouchi K., Chen Y.",Conference Paper,Scopus,chinese,[],[]
487,Exploration on Korean-Chinese collaborative tr...,In order to realize the historical sequence es...,Automatic encoder; Korean-Chinese translation;...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,309,318,10,10.1007/s00779-019-01347-5,Zhu L.,Article,Scopus,chinese,[neural],[]
1774,Description and Findings of OPPO’s Machine Tra...,This paper demonstrates our machine translatio...,Back-translation; Ensemble; Multi-task learnin...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,83,97,15,10.1007/978-981-33-6162-1_8,"Shi T., Zhang Q., Wang X., Li X., Xue Z., Hao J.",Conference Paper,Scopus,chinese,[],[]


In [7]:
GermanCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'german')]
GermanCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1045,Token reference vs. type reference: Implicatio...,This paper is concerned with the conditions un...,cognitive linguistics; English; French; generi...,https://www.scopus.com/inward/record.uri?eid=2...,1991,2.0,183,192,10,10.1007/BF00397281,Zelinsky-Wibbelt C.,Article,Scopus,german,[],[]
547,"A Network for Encoding, Decoding and Translati...",In general the spatial relationship between tw...,backpropagation; learning; locative prepositio...,https://www.scopus.com/inward/record.uri?eid=2...,1991,10.0,225,240,16,10.1080/09540099108946586,"Munro P., Tabasko M.",Article,Scopus,german,[neural],[]
874,The use of lexical semantics in interlingual m...,This paper describes the lexical-semantic basi...,abstraction; compositionality; divergences; in...,https://www.scopus.com/inward/record.uri?eid=2...,1992,28.0,135,193,59,10.1007/BF00402510,Dorr B.J.,Article,Scopus,german,[interlingual],[]
1904,Generative lexicon principles for machine tran...,This paper addresses two types of mismatches i...,compositional semantics; lexicon design; seman...,https://www.scopus.com/inward/record.uri?eid=2...,1994,4.0,155,182,28,10.1007/BF00980577,Bergler S.,Article,Scopus,german,[],[]
1902,Towards a lexicographic approach to lexical tr...,The translation of lexical items is still a fo...,German; Lexical clues; Lexical transfer; Lexic...,https://www.scopus.com/inward/record.uri?eid=2...,2001,6.0,21,87,67,10.1023/A:1013136005350,"Mel?uk I., Wanner L.",Article,Scopus,german,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,Neural machine translation with gru-gated atte...,Neural machine translation (NMT) heavily relie...,gated attention model (GAtt); Gated recurrent ...,https://www.scopus.com/inward/record.uri?eid=2...,2020,6.0,4688,4698,11,10.1109/TNNLS.2019.2957276,"Zhang B., Xiong D., Xie J., Su J.",Article,Scopus,german,[neural],[]
503,Neural machine translation with target-attenti...,"Attention mechanism, which selectively focuses...",Attention mechanism; Bidirectional targetatten...,https://www.scopus.com/inward/record.uri?eid=2...,2020,3.0,684,694,11,10.1587/transinf.2019EDP7157,"Yang M., Zhang M., Chen K., Wang R., Zhao T.",Article,Scopus,german,[neural],[]
1404,Human quality evaluation of machine-translated...,The quality of literary translation was from t...,Automatic machine translation; Domainspecific ...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,1040,1045,6,10.23919/MIPRO48935.2020.9245436,"Seljan S., Dunder I., Pavlovski M.",Conference Paper,Scopus,german,[],[]
1473,Multilingual Sentiment Analysis for Web Text B...,People's sentiments are known to have a large ...,Machine translation; Multilingual sentiment an...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,74,79,6,10.1109/IIAI-AAI50415.2020.00025,"Fujihira K., Horibe N.",Conference Paper,Scopus,german,[],[]


In [8]:
JapaneseCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'japanese')]
JapaneseCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1827,GRADE: A software environment for machine tran...,"GRADE, a software environment for machine tran...",dictionary; grammar; Japanese; Machine transla...,https://www.scopus.com/inward/record.uri?eid=2...,1988,1.0,69,82,14,10.1007/BF02057968,"Nakamura J.-i., Tsujii J.-i., Nagao M.",Review,Scopus,japanese,[],[]
1211,The treatment of complex English nominalizatio...,"This paper concerns the treatment, in the cont...",Japanese; Machine translation; nominalization;...,https://www.scopus.com/inward/record.uri?eid=2...,1988,1.0,3,21,19,10.1007/BF02057966,"Somers H., Hirakawa H., Miike S., Amano S.",Article,Scopus,japanese,[],[]
1216,Knowledge-based machine translation,This paper provides an overview of the KBMT-89...,English; interlingua; Japanese; knowledge-base...,https://www.scopus.com/inward/record.uri?eid=2...,1989,21.0,5,24,20,10.1007/BF00367750,Nirenburg S.,Article,Scopus,japanese,[],[]
1241,Automated procedures for the improvement of a ...,"In recent years, machine translation has enjoy...",English; error analysis; error correction; fee...,https://www.scopus.com/inward/record.uri?eid=2...,1990,1.0,223,246,24,10.1007/BF00389819,"Nishida F., Takamatsu S.",Article,Scopus,japanese,[],[]
1220,Relational-grammar-based generation in the JET...,This paper describes the design and function o...,generation; natural language processing; relat...,https://www.scopus.com/inward/record.uri?eid=2...,1991,1.0,1,20,20,10.1007/BF00570614,"Johnson D.E., Watanabe H.",Article,Scopus,japanese,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,Description and Findings of OPPO’s Machine Tra...,This paper demonstrates our machine translatio...,Back-translation; Ensemble; Multi-task learnin...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,83,97,15,10.1007/978-981-33-6162-1_8,"Shi T., Zhang Q., Wang X., Li X., Xue Z., Hao J.",Conference Paper,Scopus,japanese,[],[]
904,Factored phrase-based statistical machine pre-...,This paper presents the development of a casca...,Hybrid; Machine translation; Morphologically r...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,51,59,9,10.14569/IJACSA.2020.0110907,"Beyala V.L., Nkenlifack M.J., Li Litet P.",Article,Scopus,japanese,"[hybrid, statistical]",[]
1509,A Preliminary Attempt to Evaluate Machine Tran...,Providing multilingual metadata records for di...,Japanese-English; Machine translation evaluati...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,262,268,7,10.1007/978-3-030-64452-9_24,"Song Y., Batjargal B., Maeda A.",Conference Paper,Scopus,japanese,[],[]
1508,Machine Translation from Japanese to Robot Lan...,Humanoid robots should be capable of communica...,Humanoid robot; Machine translation; Natural l...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,254,260,7,10.1007/978-3-030-50726-8_33,"Manome N., Shinohara S., Suzuki K., Mitsuyoshi S.",Conference Paper,Scopus,japanese,[],[]


In [9]:
ArabicCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'arabic')]
ArabicCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1231,Lexical analysis of inflected Arabic words usi...,This paper presents a lexical analyser for inf...,Arabic; Augmented transition networks; Automat...,https://www.scopus.com/inward/record.uri?eid=2...,1993,18.0,567,588,22,10.1002/spe.4380230602,"Rafea A.A., Shaalan K.F.",Article,Scopus,arabic,[],[]
1238,A methodology for a semi-automatic evaluation ...,The lexicon is a major part of any Machine Tra...,Arabic; Coverage; Evaluation; Lexicons; Transl...,https://www.scopus.com/inward/record.uri?eid=2...,2001,4.0,127,149,23,10.1023/A:1014504808954,"Guessoum A., Zantout R.",Article,Scopus,arabic,[],[]
1106,Realization of high-performance bilingual Engl...,Document analysis and understanding (DAU) syst...,"ANN, FFT; DAU; Machine translation; OCR; Patte...",https://www.scopus.com/inward/record.uri?eid=2...,2003,1.0,54,65,12,NaN,"Arafat Ali H., El-Desouky A.I., Abd El-Gwad A.O.",Article,Scopus,arabic,[],[]
1267,A methodology for evaluating arabic machine tr...,This paper presents a methodology for evaluati...,Arabic; Coverage; Evaluation; Grammatical cove...,https://www.scopus.com/inward/record.uri?eid=2...,2004,4.0,299,335,37,10.1007/s10590-005-2412-3,"Guessoum A., Zantout R.",Article,Scopus,arabic,[],[]
1127,Empirical studies on the impact of lexical res...,"In this paper, we compile and review several e...",Bilingual lexicons; Cross-lingual retrieval; M...,https://www.scopus.com/inward/record.uri?eid=2...,2005,27.0,475,487,13,10.1016/j.ipm.2004.06.009,"Xu J., Weischedel R.",Article,Scopus,arabic,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,Improving arabic instant machine translation: ...,"Recently, instant translator applications woul...",Arabic triangle of lanuage (Almtltatallgwyh); ...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,956,965,10,10.3844/JCSSP.2020.956.965,"Albashir T.M., Alzoubi H., Albatainih M.",Article,Scopus,arabic,[],[]
1312,Enhancing Machine Translation by Integrating L...,"The word alignment process, which is a critica...",Arabic language; English language; heuristics ...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,NaN,NaN,1,10.1109/ISCV49265.2020.9204328,"Berrichi S., Mazroui A.",Conference Paper,Scopus,arabic,[],[]
1314,Machine Translation Evaluation using Textual E...,Textual entailment is a generic task aiming to...,Arabic NLP; Machine Translation Evaluation; Te...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,NaN,NaN,1,10.1109/SNAMS52053.2020.9336580,"Marouani M.E., Boudaa T., Enneya N.",Conference Paper,Scopus,arabic,[],[]
461,Unsupervised dialectal neural machine translation,"In this paper, we present the first work on un...",Neural machine translation; Regression-based d...,https://www.scopus.com/inward/record.uri?eid=2...,2020,7.0,NaN,NaN,1,10.1016/j.ipm.2019.102181,"Farhan W., Talafha B., Abuammar A., Jaikat R.,...",Article,Scopus,arabic,[neural],[]


In [11]:
ArabicCorpusCitedBy = ArabicCorpusLang.sort_values('Cited by', ascending=False)
ArabicCorpusCitedBy.reset_index(drop=True, inplace=True)
ArabicCorpusCitedBy.to_excel('ArabicCorpusCitedBy.xlsx', index=False)
ArabicCorpusCitedBy

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,Introduction to Arabic natural language proces...,This book provides system developers and resea...,Arabic; Computational linguistics; Machine tra...,https://www.scopus.com/inward/record.uri?eid=2...,2010,205.0,1,185,185,10.2200/S00277ED1V01Y201008HLT010,Habash N.Y.,Review,Scopus,arabic,[],[]
1,Hope and fear for discriminative training of s...,"In machine translation, discriminative models ...",Distributed computing; Large-margin methods; M...,https://www.scopus.com/inward/record.uri?eid=2...,2012,42.0,1159,1187,29,NaN,Chiang D.,Article,Scopus,arabic,[statistical],[]
2,Optimizing for sentence-level BLEU+1 yields sh...,We study a problem with pairwise ranking optim...,MERT; MIRA; Parameter optimization; PRO; Stati...,https://www.scopus.com/inward/record.uri?eid=2...,2012,31.0,1979,1994,16,NaN,"Nakov P., Guzmán F., Vogel S.",Conference Paper,Scopus,arabic,[statistical],[]
3,Empirical studies on the impact of lexical res...,"In this paper, we compile and review several e...",Bilingual lexicons; Cross-lingual retrieval; M...,https://www.scopus.com/inward/record.uri?eid=2...,2005,27.0,475,487,13,10.1016/j.ipm.2004.06.009,"Xu J., Weischedel R.",Article,Scopus,arabic,[],[]
4,Combining statistical translation techniques f...,Cross-language information retrieval today is ...,Context; Cross-language information retrieval;...,https://www.scopus.com/inward/record.uri?eid=2...,2012,20.0,2685,2702,18,NaN,"Ture F., Lin J., Oard D.W.",Conference Paper,Scopus,arabic,[statistical],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Towards machine comprehension of Arabic text,Machine Comprehension (MC) is a novel task of ...,Arabic Question Answering Systems; Deep Learni...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,282,288,7,10.5220/0008065402820288,"Eid A.M., El-Makky N., Nagi K.",Conference Paper,Scopus,arabic,[],[]
101,Using machine translation for automatic genre ...,This paper addresses the task of automatic gen...,Functional Text Dimensions; Genre classificati...,https://www.scopus.com/inward/record.uri?eid=2...,2018,NaN,153,162,10,NaN,"Bulygin M.V., Sharoff S.A.",Conference Paper,Scopus,arabic,[],[]
102,Transferring Informal Text in Arabic as Low Re...,Rapid growth in internet technology lead to in...,Low resource languages; Machine translation an...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,176,187,12,10.1007/978-3-030-22354-0_17,"Almansor E.H., Al-Ani A., Hussain F.K.",Conference Paper,Scopus,arabic,[],[]
103,Automatic Recognition and Translation of Polys...,In this work we study the phenomenon of verbal...,Automatic recognition and translation; Communi...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,39,51,13,10.1007/978-3-030-38833-1_4,Cheikhrouhou H.,Conference Paper,Scopus,arabic,[],[]


In [12]:
HindiCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'hindi')]
HindiCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1273,Interlingua-based English-Hindi Machine Transl...,Interlingua and transfer-based approaches to m...,Analysis; Generation; Hindi; Interlingua; Lang...,https://www.scopus.com/inward/record.uri?eid=2...,2001,44.0,251,304,54,10.1023/A:1021902704523,"Dave S., Parikh J., Bhattacharyya P.",Article,Scopus,hindi,[],[]
1170,Linguistic Resource Creation for Research and ...,Advances in statistical machine learning encou...,Cebuano; crosslanguage information retrieval; ...,https://www.scopus.com/inward/record.uri?eid=2...,2003,7.0,101,117,17,10.1145/974740.974743,"Strassel S., Maxwell M., Cieri C.",Article,Scopus,hindi,[],[]
1247,Cross-lingual C*ST*RD: English access to hindi...,"We present C*ST*RD, a cross-language informati...",Cross-language information retrieval; Headline...,https://www.scopus.com/inward/record.uri?eid=2...,2003,14.0,245,269,25,10.1145/979872.979877,"Leuski A., Lin C.-Y., Zhou L., Germann U., Och...",Article,Scopus,hindi,[],[]
1252,Making MIRACLEs: Interactive translingual sear...,Searching is inherently a user-centered proces...,Cross-language information retrieval; Interact...,https://www.scopus.com/inward/record.uri?eid=2...,2003,19.0,219,244,26,10.1145/979872.979876,"He D., Oard D.W., Wang J., Luo J., Demner-Fush...",Article,Scopus,hindi,[],[]
978,Rapid Porting of DUSTer to Hindi,The frequent occurrence of divergences-structu...,Divergences; Experimentation; Languages; machi...,https://www.scopus.com/inward/record.uri?eid=2...,2003,NaN,118,123,6,10.1145/974740.974744,"Dorr B.J., Ayan N.F., Habash N., Madnani N., H...",Article,Scopus,hindi,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,Analysing terminology translation errors in st...,Terminology translation plays a critical role ...,Machine translation; Neural machine translatio...,https://www.scopus.com/inward/record.uri?eid=2...,2020,1.0,149,195,47,10.1007/s10590-020-09251-z,"Haque R., Hasanuzzaman M., Way A.",Article,Scopus,hindi,"[neural, statistical]",[]
881,HiPHET: HYBRID APPROACH FOR TRANSLATING CODE-M...,Bilingual code-mixed (hybrid) languages have b...,Code-Mixed Language; Hinglish; HiPHET; Hybrid ...,https://www.scopus.com/inward/record.uri?eid=2...,2020,1.0,359,380,22,10.7494/CSCI.2020.21.3.3624,"Attri S.H., Prasad T.V., Ramakrishna G.",Article,Scopus,hindi,[hybrid],[]
959,Machine Translation Evaluation: Unveiling the ...,Machine Translation (MT) evaluation metrics li...,dense sentence embeddings; Evaluation of Hindi...,https://www.scopus.com/inward/record.uri?eid=2...,2020,1.0,NaN,NaN,1,10.1142/S0218001420590016,"Tripathi S., Kansal V.",Article,Scopus,hindi,[],[]
475,Neural machine translation of Hindi and English,Translation has been one of the oldest problem...,autoencoder; Bilingual embeddings; machine tra...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,2071,2079,9,10.3233/JIFS-179873,"Bhatnagar S., Chatterjee N.",Article,Scopus,hindi,[neural],[]


In [13]:
FrenchCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'french')]
FrenchCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1044,Token reference vs. type reference: Implicatio...,This paper is concerned with the conditions un...,cognitive linguistics; English; French; generi...,https://www.scopus.com/inward/record.uri?eid=2...,1991,2.0,183,192,10,10.1007/BF00397281,Zelinsky-Wibbelt C.,Article,Scopus,french,[],[]
1122,Machine translation of non-literary texts: Som...,A survey was carried out of past and current C...,ALPS; Canada; English; French; human-aided mac...,https://www.scopus.com/inward/record.uri?eid=2...,1991,NaN,21,33,13,10.1007/BF00570615,"Gotlieb C.C., d'Haenens L.",Article,Scopus,french,[],[]
1203,The lexical unit in the metal® MT system,This paper tackles some of the problems encoun...,computer-assisted translation; lexicon; Machin...,https://www.scopus.com/inward/record.uri?eid=2...,1994,1.0,1,19,19,10.1007/BF00980197,"Fontenelle T., Adriaens G., de Braekeleer G.",Article,Scopus,french,[],[]
1251,Making MIRACLEs: Interactive translingual sear...,Searching is inherently a user-centered proces...,Cross-language information retrieval; Interact...,https://www.scopus.com/inward/record.uri?eid=2...,2003,19.0,219,244,26,10.1145/979872.979876,"He D., Oard D.W., Wang J., Luo J., Demner-Fush...",Article,Scopus,french,[],[]
913,Hybrid data-driven models of machine translation,"This paper presents an extended, harmonised ac...",Chunk coverage; Convergence; Europarl corpus; ...,https://www.scopus.com/inward/record.uri?eid=2...,2005,20.0,301,323,23,10.1007/s10590-006-9015-5,"Groves D., Way A.",Conference Paper,Scopus,french,"[hybrid, statistical]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1620,Dynamic Context-guided Capsule Network for Mul...,"Multimodal machine translation (MMT), which ma...",capsule network; multimodal machine translatio...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,1320,1329,10,10.1145/3394171.3413715,"Lin H., Meng F., Su J., Yin Y., Yang Z., Ge Y....",Conference Paper,Scopus,french,[],[]
1553,Controlling Formality and Style of Machine Tra...,An often overlooked difficulty of machine tran...,Domain adaptation; Formality; Machine translation,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,306,313,8,10.1007/978-3-030-46140-9_29,"Viswanathan A., Wang V., Kononova A.",Conference Paper,Scopus,french,[],[]
1472,Multilingual Sentiment Analysis for Web Text B...,People's sentiments are known to have a large ...,Machine translation; Multilingual sentiment an...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,74,79,6,10.1109/IIAI-AAI50415.2020.00025,"Fujihira K., Horibe N.",Conference Paper,Scopus,french,[],[]
657,Transfer learning for digital heritage collect...,Transfer learning via pre-training has become ...,Cultural Heritage; Neural Machine Translation;...,https://www.scopus.com/inward/record.uri?eid=2...,2020,1.0,522,529,8,NaN,"Banar N., Lasaracina K., Daelemans W., Kestemo...",Conference Paper,Scopus,french,[neural],[]


In [14]:
SpanishCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'spanish')]
SpanishCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1167,Steps Toward Knowledge-Based Machine Translation,This paper considers the possibilities for kno...,Artificial intelligence; computational linguis...,https://www.scopus.com/inward/record.uri?eid=2...,1981,17.0,376,392,17,10.1109/TPAMI.1981.4767124,"Carbonell J.G., Cullingford R.E., Gershman A.V.",Article,Scopus,spanish,[],[]
873,The use of lexical semantics in interlingual m...,This paper describes the lexical-semantic basi...,abstraction; compositionality; divergences; in...,https://www.scopus.com/inward/record.uri?eid=2...,1992,28.0,135,193,59,10.1007/BF00402510,Dorr B.J.,Article,Scopus,spanish,[interlingual],[]
1497,A specific domain translator application in a ...,"In this paper, SAVIA is introduced; a system f...",Digital libraries; Floristic digital libraries...,https://www.scopus.com/inward/record.uri?eid=2...,2000,NaN,436,442,7,10.1007/10720076_40,"Navarrete D., Dávila R., Sánchez A.",Conference Paper,Scopus,spanish,[],[]
1250,Making MIRACLEs: Interactive translingual sear...,Searching is inherently a user-centered proces...,Cross-language information retrieval; Interact...,https://www.scopus.com/inward/record.uri?eid=2...,2003,19.0,219,244,26,10.1145/979872.979876,"He D., Oard D.W., Wang J., Luo J., Demner-Fush...",Article,Scopus,spanish,[],[]
1128,Empirical studies on the impact of lexical res...,"In this paper, we compile and review several e...",Bilingual lexicons; Cross-lingual retrieval; M...,https://www.scopus.com/inward/record.uri?eid=2...,2005,27.0,475,487,13,10.1016/j.ipm.2004.06.009,"Xu J., Weischedel R.",Article,Scopus,spanish,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,KNOWLEDGE GRAPHS EFFECTIVENESS IN NEURAL MACHI...,Maintaining semantic relations between words d...,knowledge graph representation; natural langua...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,287,306,20,10.7494/csci.2020.21.3.3701,"Ahmadnia B., Dorr B.J., Kordjamshidi P.",Article,Scopus,spanish,[neural],[]
1552,Controlling Formality and Style of Machine Tra...,An often overlooked difficulty of machine tran...,Domain adaptation; Formality; Machine translation,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,306,313,8,10.1007/978-3-030-46140-9_29,"Viswanathan A., Wang V., Kononova A.",Conference Paper,Scopus,spanish,[],[]
431,Rapid development of competitive translation e...,"Every day, more people are becoming infected a...",Automatic evaluation; COVID-19; Crisis transla...,https://www.scopus.com/inward/record.uri?eid=2...,2020,2.0,NaN,NaN,1,10.3390/informatics7020019,"Way A., Haque R., Xie G., Gaspari F., Popovi? ...",Article,Scopus,spanish,[neural],[]
1471,Multilingual Sentiment Analysis for Web Text B...,People's sentiments are known to have a large ...,Machine translation; Multilingual sentiment an...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,74,79,6,10.1109/IIAI-AAI50415.2020.00025,"Fujihira K., Horibe N.",Conference Paper,Scopus,spanish,[],[]


In [15]:
RussianCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'russian')]
RussianCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1168,Steps Toward Knowledge-Based Machine Translation,This paper considers the possibilities for kno...,Artificial intelligence; computational linguis...,https://www.scopus.com/inward/record.uri?eid=2...,1981,17.0,376,392,17,10.1109/TPAMI.1981.4767124,"Carbonell J.G., Cullingford R.E., Gershman A.V.",Article,Scopus,russian,[],[]
1151,The potential of Swetra - A multilanguage MT s...,One may indicate the potentials of an MT syste...,defective categories; English; grammatically r...,https://www.scopus.com/inward/record.uri?eid=2...,1988,5.0,237,250,14,10.1007/BF02055239,"Sigurd B., Gawronska-Werngren B.",Article,Scopus,russian,[],[]
1769,Monolingual translator workstation,Although the problem of full machine translati...,Machine translation; Monolingual user; Transla...,https://www.scopus.com/inward/record.uri?eid=2...,1998,1.0,136,149,14,10.1007/3-540-49478-2_13,"Bashkansky G., Oman U.",Conference Paper,Scopus,russian,[],[]
1264,Petr Petrovich Troyanskii (1894-1950): A forgo...,The translations of work done by Petr Petrovic...,Dictionary; History; Interlingua; Machine tran...,https://www.scopus.com/inward/record.uri?eid=2...,2000,6.0,187,221,35,10.1023/A:1011653602669,"Hutchins J., Lovtskii E.",Article,Scopus,russian,[],[]
1901,Towards a lexicographic approach to lexical tr...,The translation of lexical items is still a fo...,German; Lexical clues; Lexical transfer; Lexic...,https://www.scopus.com/inward/record.uri?eid=2...,2001,6.0,21,87,67,10.1023/A:1013136005350,"Mel?uk I., Wanner L.",Article,Scopus,russian,[],[]
1514,Cognitive Approach to Language Structure Segme...,The problem of establishing transferable langu...,Machine translation; Phrase structures; Semant...,https://www.scopus.com/inward/record.uri?eid=2...,2003,12.0,49,55,7,NaN,Kozerenko E.B.,Conference Paper,Scopus,russian,[],[]
1499,Syntactic polysemy in machine translation,The paper focuses on the issues of establishin...,Disambiguation; Linear structures; Machine tra...,https://www.scopus.com/inward/record.uri?eid=2...,2004,1.0,807,813,7,NaN,Kozerenko E.B.,Conference Paper,Scopus,russian,[],[]
1427,Improved word alignments using the Web as a co...,We propose a novel method for improving word a...,Competitive linking; Edit distance; Machine tr...,https://www.scopus.com/inward/record.uri?eid=2...,2007,3.0,400,405,6,NaN,"Nakov P., Nakov S., Paskaleva E.",Conference Paper,Scopus,russian,[],[]
153,Multi-alignment templates induction,This paper examins approaches for translation ...,A; EM algorithm; Machine translation; Natural ...,https://www.scopus.com/inward/record.uri?eid=2...,2008,4.0,535,554,20,NaN,"Laukaitis A., Vasilecas O.",Article,Scopus,russian,[statistical],[]
238,Unsupervised extraction of false friends from ...,False friends are pairs of words in two langua...,Cognates; Cross-lingual semantic similarity; F...,https://www.scopus.com/inward/record.uri?eid=2...,2009,1.0,292,298,7,NaN,"Nakov S., Nakov P., Paskaleva E.",Conference Paper,Scopus,russian,[statistical],[]


In [16]:
KoreanCorpusLang = LongCorpusLang.loc[(LongCorpusLang.astype(str)['From-->Too'] == 'korean')]
KoreanCorpusLang.sort_values(by=['Year'])

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
1052,Disambiguating word senses in Korean-Japanese ...,This paper presents a method for disambiguatin...,Corpus analysis; Machine translation; Ontology...,https://www.scopus.com/inward/record.uri?eid=2...,2002,1.0,1688,1697,10,NaN,"Kang S.-J., Chung Y.-J., Lee J.-H.",Article,Scopus,korean,[],[]
1452,"Treating ""hata""-construction in Korean-Chinese MT",This paper addresses the treatment of light-ve...,'Hata' Verb; Korean-Chinese Machine Translatio...,https://www.scopus.com/inward/record.uri?eid=2...,2003,1.0,375,380,6,NaN,"Seo Y.-A., Huang Y., Hong M., Choi S.-K.",Conference Paper,Scopus,korean,[],[]
1574,A method for English-Korean target word select...,Target word selection is one of the most impor...,Machine translation; Target word selection; Wo...,https://www.scopus.com/inward/record.uri?eid=2...,2006,7.0,1622,1629,8,10.1093/ietfec/e89-a.6.1622,"Lee K.-Y., Park S.-K., Kim H.-W.",Conference Paper,Scopus,korean,[],[]
1709,Translation of unknown terms via web mining fo...,Many English words appear in Asian language te...,Cross-language information retrieval; Indexing...,https://www.scopus.com/inward/record.uri?eid=2...,2006,1.0,258,269,12,10.1007/11880592_20,"Li Q., Myaeng S.H., Jin Y., Kang B.-Y.",Conference Paper,Scopus,korean,[],[]
1021,An alignment model for extracting English-Kore...,Technical terms are linguistic representations...,Alignment; Machine translation; Technical term...,https://www.scopus.com/inward/record.uri?eid=2...,2006,NaN,2972,2980,9,10.1093/ietisy/e89-d.12.2972,"Oh J.-H., Choi K.-S., Isahara H.",Article,Scopus,korean,[],[]
1598,The role of annotation in intercultural commun...,"In intercultural communication, there are larg...",Annotation; Computer-mediated communication; I...,https://www.scopus.com/inward/record.uri?eid=2...,2007,5.0,186,195,10,10.1007/978-3-540-73287-7_24,"Shigenobu T., Fujii K., Yoshino T.",Conference Paper,Scopus,korean,[],[]
1058,Customizing an English-Korean machine translat...,This paper addresses a method for customizing ...,Customization; Machine translation; Patent mac...,https://www.scopus.com/inward/record.uri?eid=2...,2007,2.0,105,114,10,NaN,"Choi S.-K., Kim Y.-G.",Article,Scopus,korean,[],[]
1073,Intra-sentence segmentation based on support v...,This work is about intra-sentence segmentation...,Gaussian kernel; Intra-sentence segmentation; ...,https://www.scopus.com/inward/record.uri?eid=2...,2008,2.0,2673,2682,10,10.1016/j.eswa.2007.05.032,"Kim Y.-S., Oh Y.-J.",Article,Scopus,korean,[],[]
1652,What is needed the most in MT-supported paper ...,This paper addresses our system which provides...,Korean-English translation; Machine translatio...,https://www.scopus.com/inward/record.uri?eid=2...,2008,NaN,418,427,10,NaN,"Kim C.H., Kwon O.-W., Kim Y.K.",Conference Paper,Scopus,korean,[],[]
1544,How to overcome the domain barriers in pattern...,One of difficult issues in pattern-based machi...,Customization; Machine translation; Pattern-ba...,https://www.scopus.com/inward/record.uri?eid=2...,2008,NaN,161,168,8,NaN,"Choi S.-K., Lee K.-Y., Roh Y.-H., Kwon O.-W., ...",Conference Paper,Scopus,korean,[],[]


### Methodes Separation

In [17]:
LongCorpusMethod = Corpus.explode('Method Used').reset_index(drop=True)
print('the lenght of the corpus after separating the languages are : '+str(len(LongCorpusMethod)))
LongCorpusMethod.head()

the lenght of the corpus after separating the languages are : 2093


,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,"[persian, arabic, english]",statistical,[]
1,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,"[arabic, english]",statistical,[]
2,An automatic and a machine-assisted method to ...,Two different methods of corpus cleaning are p...,Bilingual corpus cleaning; Statistical machine...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3342351,"Srivastava J., Sanyal S., Srivastava A.K.",Article,Scopus,"[hindi, english]",statistical,[]
3,Combining MEDLINE and publisher data to create...,Background: Most of the institutional and rese...,Biomedical domain; Multilingual corpus generat...,https://www.scopus.com/inward/record.uri?eid=2...,2013,14.0,NaN,NaN,1,10.1186/1471-2105-14-146,"Jimeno Yepes A., Prieur-Gaston É., Névéol A.",Article,Scopus,"[spanish, french, english]",statistical,[]
4,Syntax-based Chinese-Vietnamese tree-to-tree s...,"Because of the scarcity of bilingual corpora, ...",Chinese-Vietnamese; Linguistic features; Pruni...,https://www.scopus.com/inward/record.uri?eid=2...,2019,1.0,NaN,NaN,1,10.1145/3314938,"Gao S., Huang J., Xue M., Yu Z., Wang Z., Zhan...",Article,Scopus,"[vietnamese, chinese]",statistical,[]


In [18]:
NeuralCorpusMethod = LongCorpusMethod.loc[(LongCorpusMethod.astype(str)['Method Used'] == 'neural')]
#NeuralCorpusMethod.sort_values(by=['Cited By'])

NeuralCorpusCitedBy = NeuralCorpusMethod.sort_values('Cited by', ascending=False)
NeuralCorpusCitedBy.reset_index(drop=True, inplace=True)
NeuralCorpusCitedBy.to_excel('NeuralCorpusCitedBy.xlsx', index=False)
NeuralCorpusCitedBy

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,ProLanGO: Protein function prediction using ne...,With the development of next generation sequen...,Machine learning; Neural machine translation; ...,https://www.scopus.com/inward/record.uri?eid=2...,2017,85.0,NaN,NaN,1,10.3390/molecules22101732,"Cao R., Freitas C., Chan L., Sun M., Jiang H.,...",Article,Scopus,[],neural,[]
1,Understanding subtitles by character-level seq...,This paper presents a character-level seque-nc...,Character-level; neural machine translation; r...,https://www.scopus.com/inward/record.uri?eid=2...,2017,77.0,616,624,9,10.1109/TII.2016.2601521,"Zhang H., Li J., Ji Y., Yue H.",Article,Scopus,"[chinese, english]",neural,[]
2,Fine-grained attention mechanism for neural ma...,Neural machine translation (NMT) has been a ne...,Attention mechanism; Fine-grained attention; N...,https://www.scopus.com/inward/record.uri?eid=2...,2018,45.0,171,176,6,10.1016/j.neucom.2018.01.007,"Choi H., Cho K., Bengio Y.",Article,Scopus,[],neural,[]
3,A Hierarchy-to-Sequence Attentional Neural Mac...,Although sequence-to-sequence attentional neur...,Hierarchy-to-sequence; natural language proces...,https://www.scopus.com/inward/record.uri?eid=2...,2018,41.0,623,632,10,10.1109/TASLP.2018.2789721,"Su J., Zeng J., Xiong D., Liu Y., Wang M., Xie J.",Article,Scopus,"[chinese, german, english]",neural,[]
4,Neural-machine-translation-based commit messag...,Commit messages can be regarded as the documen...,Commit message generation; Nearest neighbor al...,https://www.scopus.com/inward/record.uri?eid=2...,2018,38.0,373,384,12,10.1145/3238147.3238190,"Liu Z., Lo D., Xia X., Xing Z., Hassan A.E., W...",Conference Paper,Scopus,[],neural,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,Iterative training of unsupervised neural and ...,Recent work achieved remarkable results in tra...,Knowledge acquisition; Low-resource language p...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,NaN,NaN,1,10.1145/3389790,"Marie B., Fujita A.",Article,Scopus,[],neural,[]
443,Neural machine translation and statistical mac...,The machine translation field has changed comp...,Edit distance; Neural Machine Translation; Per...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,1,19,19,10.5565/rev/tradumatica.235,Pereira A.L.,Article,Scopus,[],neural,[]
444,Self-attention and dynamic convolution hybrid ...,"In sequence-to-sequence learning, models based...",Convolution; NMT; Self-attention; Seq2seq,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,352,359,8,10.1109/ICBK50248.2020.00057,"Zhang Z., Wu S., Chen G., Jiang D.",Conference Paper,Scopus,"[german, english]",neural,[]
445,Towards hybrid neural machine translation for ...,This paper investigates a hybrid method for tr...,English; Hybrid system; Latvian; Neural machin...,https://www.scopus.com/inward/record.uri?eid=2...,2016,NaN,84,91,8,10.3233/978-1-61499-701-6-84,Pinnis M.,Conference Paper,Scopus,[english],neural,[]


In [19]:
NeuralArabicCorpus = pd.merge(NeuralCorpusCitedBy, ArabicCorpusCitedBy, how='inner', on=['DOI'])
NeuralArabicCorpus.to_excel('NeuralArabicCorpus.xlsx', index=False)
NeuralArabicCorpus

,Title_x,Abstract_x,Author Keywords_x,Link_x,Year_x,Cited by_x,Page start_x,Page end_x,Page count_x,DOI,...,Cited by_y,Page start_y,Page end_y,Page count_y,Authors_y,Document Type_y,Source_y,From-->Too_y,Method Used_y,is it a survey?_y
0,Neural machine translation tools in the langua...,Web-based machine translation (wbmt) tools hav...,Analysis of translator output; English as a fo...,https://www.scopus.com/inward/record.uri?eid=2...,2018,14.0,3,24,22,NaN,...,42.0,1159,1187,29,Chiang D.,Article,Scopus,arabic,[statistical],[]
1,Neural machine translation tools in the langua...,Web-based machine translation (wbmt) tools hav...,Analysis of translator output; English as a fo...,https://www.scopus.com/inward/record.uri?eid=2...,2018,14.0,3,24,22,NaN,...,31.0,1979,1994,16,"Nakov P., Guzmán F., Vogel S.",Conference Paper,Scopus,arabic,[statistical],[]
2,Neural machine translation tools in the langua...,Web-based machine translation (wbmt) tools hav...,Analysis of translator output; English as a fo...,https://www.scopus.com/inward/record.uri?eid=2...,2018,14.0,3,24,22,NaN,...,20.0,2685,2702,18,"Ture F., Lin J., Oard D.W.",Conference Paper,Scopus,arabic,[statistical],[]
3,Neural machine translation tools in the langua...,Web-based machine translation (wbmt) tools hav...,Analysis of translator output; English as a fo...,https://www.scopus.com/inward/record.uri?eid=2...,2018,14.0,3,24,22,NaN,...,4.0,396,405,10,"Akeel M., Mishra R.",Article,Scopus,arabic,[neural],[]
4,Neural machine translation tools in the langua...,Web-based machine translation (wbmt) tools hav...,Analysis of translator output; English as a fo...,https://www.scopus.com/inward/record.uri?eid=2...,2018,14.0,3,24,22,NaN,...,3.0,271,277,7,"Ahmed S.A., Joshi N., Mathur I., Katyayan P.",Article,Scopus,arabic,[statistical],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,Domain adaptation using neural network joint m...,We explore neural joint models for the task of...,Distributed representation of texts; Domain ad...,https://www.scopus.com/inward/record.uri?eid=2...,2017,2.0,161,179,19,10.1016/j.csl.2016.12.006,...,2.0,161,179,19,"Joty S., Durrani N., Sajjad H., Abdelali A.",Article,Scopus,arabic,[neural],[]
612,Improving Arabic neural machine translation vi...,Even though the rise of the neural machine tra...,Machine translation; Natural language processi...,https://www.scopus.com/inward/record.uri?eid=2...,2019,2.0,279,314,36,10.1007/s10590-019-09237-6,...,2.0,279,314,36,"Hadj Ameur M.S., Guessoum A., Meziane F.",Article,Scopus,arabic,[neural],[]
613,Chinese-Catalan: A neural machine translation ...,This article innovatively addresses machine tr...,Chinese-Catalan; Neural machine translation; P...,https://www.scopus.com/inward/record.uri?eid=2...,2019,1.0,NaN,NaN,1,10.1145/3312575,...,1.0,NaN,NaN,1,"Costa-Jussà M.R., Casas N., Escolano C., Fonol...",Article,Scopus,arabic,[neural],[]
614,Paraphrasing Arabic Metaphor with Neural Machi...,The task of recognizing and generating paraphr...,Arabic language; Metaphor; Neural Machine Tran...,https://www.scopus.com/inward/record.uri?eid=2...,2018,1.0,308,314,7,10.1016/j.procs.2018.10.493,...,1.0,308,314,7,"Alkhatib M., Shaalan K.",Conference Paper,Scopus,arabic,[neural],[]


### explode methods and languages and take only the english and arabic languages where the neural method

In [21]:
CorpusMethodLanguage = Corpus.explode('Method Used').reset_index(drop=True).explode('From-->Too').reset_index(drop=True)
print('the lenght of the corpus after separating the languages and methods are : '+str(len(CorpusMethodLanguage)))
CorpusMethodLanguage.head()

the lenght of the corpus after separating the languages and methods are : 3573


,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,persian,statistical,[]
1,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,arabic,statistical,[]
2,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,english,statistical,[]
3,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,arabic,statistical,[]
4,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,english,statistical,[]


In [26]:
CorpusMethodLanguage.drop(CorpusMethodLanguage.index[(CorpusMethodLanguage['Year'] == 2021.0) | (CorpusMethodLanguage['Year'] == 2022.0)], inplace = True)
ArabicNeuralCorpus = CorpusMethodLanguage.loc[(CorpusMethodLanguage.astype(str)['From-->Too'] == 'arabic') & (CorpusMethodLanguage.astype(str)['Method Used'] == 'neural')]

ArabicNeuralCorpus.reset_index(drop=True, inplace=True)
ArabicNeuralCorpus.to_excel('ArabicNeuralCorpus.xlsx', index=False)
ArabicNeuralCorpus

,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,Chinese-Catalan: A neural machine translation ...,This article innovatively addresses machine tr...,Chinese-Catalan; Neural machine translation; P...,https://www.scopus.com/inward/record.uri?eid=2...,2019,1.0,NaN,NaN,1,10.1145/3312575,"Costa-Jussà M.R., Casas N., Escolano C., Fonol...",Article,Scopus,arabic,neural,[]
1,Addressing data sparsity for neural machine tr...,Translating between morphologically rich langu...,Deep learning; Factored models; Neural machine...,https://www.scopus.com/inward/record.uri?eid=2...,2020,2.0,NaN,NaN,1,10.1007/s10590-019-09242-9,"García-Martínez M., Aransa W., Bougares F., Ba...",Article,Scopus,arabic,neural,[]
2,Unsupervised dialectal neural machine translation,"In this paper, we present the first work on un...",Neural machine translation; Regression-based d...,https://www.scopus.com/inward/record.uri?eid=2...,2020,7.0,NaN,NaN,1,10.1016/j.ipm.2019.102181,"Farhan W., Talafha B., Abuammar A., Jaikat R.,...",Article,Scopus,arabic,neural,[]
3,ANN and rule based method for English to Arabi...,Machine translation is the process by which co...,Back-propagation; English-Arabic machine trans...,https://www.scopus.com/inward/record.uri?eid=2...,2014,4.0,396,405,10,NaN,"Akeel M., Mishra R.",Article,Scopus,arabic,neural,[]
4,Sentence alignment using feed forward neural n...,Parallel corpora have become an essential reso...,English/Arabic parallel corpus; Machine transl...,https://www.scopus.com/inward/record.uri?eid=2...,2006,3.0,423,434,12,10.1142/S0129065706000822,"Fattah M.A., Ren F., Kuroiwa S.",Article,Scopus,arabic,neural,[]
5,Arabic-Chinese Neural Machine Translation: Rom...,Morphologically rich and complex languages suc...,Arabic morphology; Arabic romanization; BPE; d...,https://www.scopus.com/inward/record.uri?eid=2...,2019,4.0,133122,133135,14,10.1109/ACCESS.2019.2941161,"Aqlan F., Fan X., Alqwbani A., Al-Mansoub A.",Article,Scopus,arabic,neural,[]
6,Domain adaptation using neural network joint m...,We explore neural joint models for the task of...,Distributed representation of texts; Domain ad...,https://www.scopus.com/inward/record.uri?eid=2...,2017,2.0,161,179,19,10.1016/j.csl.2016.12.006,"Joty S., Durrani N., Sajjad H., Abdelali A.",Article,Scopus,arabic,neural,[]
7,Improving Arabic neural machine translation vi...,Even though the rise of the neural machine tra...,Machine translation; Natural language processi...,https://www.scopus.com/inward/record.uri?eid=2...,2019,2.0,279,314,36,10.1007/s10590-019-09237-6,"Hadj Ameur M.S., Guessoum A., Meziane F.",Article,Scopus,arabic,neural,[]
8,Paraphrasing Arabic Metaphor with Neural Machi...,The task of recognizing and generating paraphr...,Arabic language; Metaphor; Neural Machine Tran...,https://www.scopus.com/inward/record.uri?eid=2...,2018,1.0,308,314,7,10.1016/j.procs.2018.10.493,"Alkhatib M., Shaalan K.",Conference Paper,Scopus,arabic,neural,[]
9,Building a neural network-based English-to-Ara...,This paper presents a Transfer Module for an E...,Arabic; Bilingual corpora; Machine translation...,https://www.scopus.com/inward/record.uri?eid=2...,2010,3.0,94,101,8,10.1109/ICMWI.2010.5648157,"Al Dam R., Guessoum A.",Conference Paper,Scopus,arabic,neural,[]
